In [3]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import pandas as pd

import nltk

import matplotlib.pyplot as plt

## Get and preprocess the data

In [4]:
df = pd.read_feather("../data/twitter_mutlimodal_hate_speech/data.feather")

In [5]:
labels = df["labels_str"].apply(pd.Series)

display(labels.head(5))
df.loc[16833, "tweet_text"]

# df[labels.columns] =  labels


display(pd.concat([labels[col] for col in labels.columns]).value_counts())


# One-hot encoded categories
# pd.get_dummies(labels)
oh_labels = pd.concat([
    pd.get_dummies(labels[li])
    for li in labels.columns
]).groupby(level=0).max().astype(bool)

df[oh_labels.columns] = oh_labels

display(df.head())

,0,1,2,3,4
0,Religion,Racist,Homophobe,NaN,NaN
1,OtherHate,OtherHate,OtherHate,NaN,NaN
2,NotHate,NotHate,NotHate,NaN,NaN
3,Racist,NotHate,NotHate,NaN,NaN
4,Racist,NotHate,Racist,NaN,NaN


NotHate      312039
Racist        63543
OtherHate     31548
Sexist        22805
Homophobe     16932
Religion       2607
dtype: int64

,img_url,labels,tweet_url,tweet_text,labels_str,tweet_id,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist
0,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]",1114679353714016256,True,False,False,True,True,False
1,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]",1063020048816660480,False,False,True,False,False,False
2,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]",1108927368075374593,False,True,False,False,False,False
3,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]",1114558534635618305,False,True,False,True,False,False
4,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]",1035252480215592966,False,True,False,True,False,False


In [6]:
# Homophobe = df.loc[df["Homophobe"], "tweet_text"]
# NotHate = df.loc[df["NotHate"], "tweet_text"]
# OtherHate = df.loc[df["OtherHate"], "tweet_text"]
# Racist = df.loc[df["Racist"], "tweet_text"]
# Religion = df.loc[df["Religion"], "tweet_text"]
# Sexist = df.loc[df["Sexist"], "tweet_text"]

### Cleaning text

In [7]:
def clean_tweets(vTEXT):
    """
    Removes URLs, tags and more from tweets. Extend it if needed.
    
    short words: \b\w{1,2}\b
    tags starting with @: @\w*
    URL: (https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b
    """
    vTEXT = re.sub(r'\b\w{1,2}\b|@\w*|(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

# Racist = Racist.apply(remove_urls)

In [8]:
data = df.copy()

data["text"] = data["tweet_text"].apply(clean_tweets)
data = data[["text", "Homophobe", "NotHate", "OtherHate", "Racist", "Religion", "Sexist"]]

# data = data[["text", "Homophobe", "NotHate", "OtherHate", "Racist", "Religion", "Sexist"]].melt(
# #     id_vars="tweet_text"
#     id_vars=['text'], value_vars=[
#         "Homophobe",
#         "NotHate",
#         "OtherHate",
#         "Racist",
#         "Religion",
#         "Sexist"
#     ],
#     var_name="category"
# )

# data = data[data["value"] & (data["category"] != "NotHate")].drop(columns=("value"))

In [9]:
data

,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist
0,Nigga,True,False,False,True,True,False
1,horses are retarded,False,False,True,False,False,False
2,“NIGGA MOMMA YOUNGBOY SPITTING REAL SHIT NI...,False,True,False,False,False,False
3,xxSuGVNGxx: ran into this HOLY NIGGA TODAY 😭...,False,True,False,True,False,False
4,“EVERYbody calling you Nigger now!”,False,True,False,True,False,False
...,...,...,...,...,...,...,...
149818,would just say hes Donny the retard,False,True,True,False,False,True
149819,congrats nigga keep grinding,False,True,False,False,False,False
149820,nigga big shitty,False,True,False,True,False,False
149821,did she just say “ nigga” Rich? &amp; she sai...,False,True,False,True,False,False


### Stemming

In [10]:
# Load SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token.
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # Filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation).
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # First tokenize by sentence, then by word to ensure that punctuation is caught as it's own token.
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # Filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation).
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [11]:
tweets_text = data["text"].tolist()

In [12]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in tweets_text:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

# Create a pandas DataFrame
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

print("There are '{}' items in our data frame.".format(str(vocab_frame.shape[0])))
print("Data frame contents: \n{}".format(vocab_frame.head(10)))

There are '1146606' items in our data frame.
Data frame contents: 
             words
nigga        nigga
hors        horses
are            are
retard    retarded
nigga        nigga
momma        momma
youngboy  youngboy
spit      spitting
real          real
shit          shit


## TF-IDF Encoding

In [13]:
# Define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
                        max_df=0.8, 
                        max_features=200000,
#                         min_df=0.2, 
                        min_df=0.0015,
#                         min_df=0.002,
                        stop_words='english', 
                        use_idf=True, 
                        tokenizer=tokenize_and_stem, 
                        ngram_range=(1,3))

# Fit the vectorizer to synopses texts
tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_text) 

print("TF-IDF matrix shape: {}".format(tfidf_matrix.shape))

/home/tomaz/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


TF-IDF matrix shape: (149823, 589)


In [14]:
print("TF-IDF vectors (each column is a document):\n{}\nRows:\n{}".format(tfidf_matrix.T.A, tfidf_vectorizer.get_feature_names()))

TF-IDF vectors (each column is a document):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Rows:
['absolut', 'account', 'act', 'actual', 'ago', 'ain', 'aint', 'alreadi', 'alway', 'america', 'amp', 'ani', 'anim', 'anoth', 'anoth nigga', 'anyon', 'anyth', 'asf', 'asian', 'ask', 'ass', 'ass bitch', 'ass bitch fuck', 'ass nigga', 'away', 'aye', 'babe', 'babi', 'bad', 'bag', 'ball', 'bar', 'beat', 'becaus', 'befor', 'believ', 'best', 'bet', 'better', 'big', 'big dyke', 'big dyke energi', 'bint', 'birthday', 'birthday nigga', 'bit', 'bitch', 'bitch ass', 'bitch ass nigga', 'bitch fuck', 'bitch fuck bout', 'bitch nigga', 'black', 'bless', 'block', 'blond', 'bodi', 'book', 'border', 'bout', 'bout nigga', 'boy', 'break', 'bring', 'bro', 'broke', 'broke nigga', 'brother', 'bruh', 'buildthewal', 'buildthewal maga', 'busi', 'buy', 'came', 'car', 'card', 'care', 'cat', 'catch', 'caus', 'chan

In [15]:
terms = tfidf_vectorizer.get_feature_names()

# from sklearn.metrics.pairwise import cosine_similarity
# dist = 1 - cosine_similarity(tfidf_matrix)

## Clustering

In [16]:
num_clusters = 10
km = KMeans(n_clusters=num_clusters)

# Perform clustering
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
print("Clusters: {}".format(clusters))

Clusters: [2, 6, 0, 0, 3, 0, 0, 4, 8, 2, 0, 3, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 7, 5, 5, 5, 0, 0, 0, 4, 8, 0, 0, 8, 0, 0, 5, 0, 0, 4, 0, 5, 0, 0, 0, 0, 3, 0, 6, 5, 0, 0, 0, 0, 0, 0, 6, 6, 0, 5, 5, 0, 0, 0, 3, 6, 0, 7, 4, 5, 0, 3, 3, 0, 6, 7, 4, 8, 1, 0, 0, 0, 0, 0, 0, 2, 1, 4, 5, 0, 6, 4, 0, 0, 5, 0, 5, 2, 0, 0, 0, 1, 9, 8, 0, 0, 8, 0, 6, 0, 0, 0, 2, 7, 1, 0, 1, 4, 5, 0, 0, 2, 2, 0, 6, 0, 0, 0, 6, 0, 0, 0, 5, 6, 0, 2, 3, 0, 4, 6, 4, 3, 4, 7, 0, 0, 1, 0, 9, 0, 0, 0, 0, 2, 0, 0, 7, 0, 0, 5, 0, 3, 0, 0, 2, 0, 8, 2, 0, 7, 5, 6, 8, 0, 0, 0, 3, 9, 6, 3, 6, 0, 0, 0, 3, 0, 4, 0, 0, 7, 0, 0, 6, 5, 0, 4, 0, 4, 8, 0, 0, 0, 6, 0, 6, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 8, 0, 6, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 8, 0, 7, 0, 0, 0, 0, 7, 2, 0, 0, 0, 4, 0, 0, 8, 4, 0, 7, 2, 9, 0, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 5, 2, 0, 0, 2, 6, 1, 4, 7, 0, 4, 7, 0, 0, 0, 0, 6, 3, 6, 0, 5, 0, 4, 0, 0, 0, 3, 0, 5, 6, 2, 0, 5, 0, 6, 5, 2, 0, 0, 5, 0, 8, 0, 2, 0, 0, 5, 4, 6, 0, 3, 4, 0, 7, 0, 2, 0,

In [17]:
# # Elbow method
# distortions = []
# for i in range(1, 20):
#     km = KMeans(
#         n_clusters=i, init='random',
#         n_init=10, max_iter=300,
#         tol=1e-04, random_state=0
#     )
#     km.fit(tfidf_matrix)
#     distortions.append(km.inertia_)

# # plot
# plt.plot(range(1, 20), distortions, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Distortion')
# plt.show()

In [18]:
# import joblib

# Uncomment the below line to save your model 
#joblib.dump(km,  'doc_cluster.pkl')

# Uncomment the below line to load your saved model 
#km = joblib.load('doc_cluster.pkl')
#clusters = km.labels_.tolist()
# print("Clusters: {}".format(clusters))

In [19]:
data["cluster"] = clusters

print("Number of movies per cluster: \n{}".format(data["cluster"].value_counts()))

Number of movies per cluster: 
0    86169
4    13580
2    11692
6     8707
5     7575
7     6569
8     5667
3     4744
9     2917
1     2203
Name: cluster, dtype: int64


In [20]:
# data.groupby("cluster")["category"].value_counts()

## Researching clusters

In [28]:
print("Top terms per cluster:\n")

# Sort cluster centers by proximity to centroid.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

# Helper function
def getClusterWords(cluster, n=15):
    words = []
    for ind in order_centroids[cluster, :n]: # Print 6 words per cluster
        print(terms[ind])
        words.append(vocab_frame.loc[terms[ind].split(' '),].values.tolist()[0][0])
    return ", ".join(words)

for i in range(num_clusters):
    print("-"*50)
    print(f"Cluster {i}")
    
    print(getClusterWords(i))
    
    t = data[data["cluster"] == i]
    display(t)
    # Note that this does NOT add to 1, as one tweet has multiple categories
    display((t.drop(columns=["text", "cluster"]).sum() / len(t)).round(3))
    
#     break

    print("\n")

Top terms per cluster:

--------------------------------------------------
Cluster 0
nigga
like
got
ass
fuck
just
surrend
said
shit
look
real
buildthewal
hillbilli
white
don
nigga, like, got, ass, fuck, just, surrender, said, shit, look, real, buildthewall, hillbilly, white, don


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
2,“NIGGA MOMMA YOUNGBOY SPITTING REAL SHIT NI...,False,True,False,False,False,False,0
3,xxSuGVNGxx: ran into this HOLY NIGGA TODAY 😭...,False,True,False,True,False,False,0
5,“ real ass bitch give fuck boutta nigga”,False,True,False,False,False,False,0
6,Fuck ice. White supremacist trash. All you ...,False,False,True,True,False,False,0
10,.... you turn twitter for instead beating ...,False,True,False,False,False,False,0
...,...,...,...,...,...,...,...,...
149815,know ’ too much real nigga ever danger ...,False,True,False,False,False,False,0
149816,Good luck nigga 🤘🏾,False,True,False,True,False,False,0
149820,nigga big shitty,False,True,False,True,False,False,0
149821,did she just say “ nigga” Rich? &amp; she sai...,False,True,False,True,False,False,0


Homophobe    0.027
NotHate      0.970
OtherHate    0.100
Racist       0.395
Religion     0.016
Sexist       0.069
dtype: float64



--------------------------------------------------
Cluster 1
na
wan
wan na
gon na
gon
nigga
nigga gon
just
like
fuck
don
say
make
cunt
ass
na, wan, wan, gon, gon, nigga, nigga, just, like, fuck, don, says, make, cunt, ass


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
17,When android nigga wanna talk about phone bat...,False,True,False,False,False,False,1
79,Wanna see slimy twat? 👇,False,True,True,False,False,True,1
87,LEMONADE NIGGA WANNA DRANK 🤪,False,True,False,True,False,True,1
102,“How this nigga gonna tell drug free?”,False,True,False,True,False,False,1
115,yes they can say dyke what gonna about huh,False,True,False,False,False,False,1
...,...,...,...,...,...,...,...,...
149451,Nigga look like wanna home 😭,False,True,False,False,False,False,1
149621,"Someone wanna Text something, nigga bored",False,True,False,True,False,False,1
149623,see this faggot junk kid one more time ’ gon...,True,True,False,False,False,False,1
149733,just wanna get one more bag nigga...,False,True,False,False,False,False,1


...

,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
8,#sissy faggot,True,True,False,False,False,False,8
31,and are the Jack and Ennis from Brokeback (f...,True,False,False,True,False,False,8
34,"fucked , response was too faggot like, need...",True,True,False,False,False,False,8
78,proud #serve the #superior #alphamale pro...,False,True,False,False,False,False,8
104,Faggot,True,False,False,False,False,False,8
...,...,...,...,...,...,...,...,...
149685,when see faggot floor,True,True,True,False,False,False,8
149716,The nerve this Faggot make this statement!,True,True,True,False,False,False,8
149749,Relapsing Faggot..... #findom # piggy #paypig,True,True,False,False,False,False,8
149769,faggot need attention,True,False,True,False,False,False,8


Homophobe    0.779
NotHate      0.766
OtherHate    0.161
Racist       0.100
Religion     0.014
Sexist       0.149
dtype: float64



--------------------------------------------------
Cluster 9
redneck
countri
just
like
love
omg
look
new
book
amp
speak
win
day
time
white
redneck, country, just, like, love, omg, look, new, book, amp, speak, win, day, time, white


,text,Homophobe,NotHate,OtherHate,Racist,Religion,Sexist,cluster
103,Another benefit being Redneck you are not e...,False,True,True,True,False,False,9
149,Picked the coolest tequila miniatures from CD...,False,True,False,False,False,False,9
178,Redneck #,False,False,True,True,False,False,9
251,Just another day the trailer park. Shout out ...,False,False,False,True,False,False,9
368,The Most Redneck Cities Massachusetts Explai...,False,True,False,False,False,False,9
...,...,...,...,...,...,...,...,...
149325,Yay redneck pants 👍,False,True,False,True,False,False,9
149419,You might redneck you had this for lunch,False,True,False,False,False,False,9
149435,Hot Str8 Redneck Guy Jerks Off With And Shoot...,False,True,True,False,False,False,9
149580,Redneck Wizard Selling His Homemade Steel-Toe...,False,True,False,False,False,False,9


Homophobe    0.031
NotHate      0.929
OtherHate    0.360
Racist       0.459
Religion     0.020
Sexist       0.042
dtype: float64